<a href="https://colab.research.google.com/github/jux310/CombinadorUade/blob/main/Uade_Scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import difflib  # Para la comparación aproximada

# URL principal de la página a extraer datos
url = "https://www.webcampus.uade.edu.ar/MiProgramacion/Contenidos/MiProgramacion.aspx?IdUsuario=br+B3VmrZbw="

# URL de filtro (si se deja vacío, no se aplica filtro)
url_filtro = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQ5Rw3QfNhw-TghB1XV3FMLjPsN9EKgr4e2Z0Awz169kXXB940iORYEwWE8RhiNeDmfgSFFyHPGZrh6/pub?gid=813457300&single=true&output=csv"
# url_filtro = ""  # Descomentar esta línea para no aplicar filtro

http_headers = {
    "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/91.0.4472.124 Safari/537.36"),
    "Referer": url
}

def eliminar_tildes(texto):
    return ''.join(
        char for char in unicodedata.normalize('NFKD', str(texto))
        if not unicodedata.combining(char)
    )

def get_form_fields(soup):
    """
    Recolecta todos los campos del formulario, incluyendo inputs hidden y selects.
    """
    fields = {}
    # Inputs hidden
    for input_tag in soup.find_all('input', type='hidden'):
        name = input_tag.get('name')
        if name:
            fields[name] = input_tag.get('value', '')
    # Selects (dropdowns)
    for select_tag in soup.find_all('select'):
        name = select_tag.get('name')
        if name:
            selected_option = select_tag.find('option', selected=True)
            if selected_option:
                fields[name] = selected_option.get('value', '')
            else:
                fields[name] = ''
    return fields

def parse_table(soup):
    """
    Extrae los datos de la tabla.
    Primero intenta buscar la tabla por el id esperado, y si no la encuentra,
    busca directamente en el <tbody>.
    Para las columnas Col9 a Col15 (índices 8 a 14) se asigna True si el estilo contiene
    "background-color:LightGrey", o False en caso contrario.
    Solo se procesan filas que tengan 18 columnas.
    """
    # Intentar encontrar la tabla por id
    table = soup.find('table', {'id': 'ctl00_ContentPlaceHolderMain_RadGrid1_ctl00'})
    if table:
        tbody = table.find('tbody')
    else:
        tbody = soup.find('tbody')

    if not tbody:
        print("No se encontró el <tbody> en el HTML.")
        return [], []

    rows = []
    for row in tbody.find_all('tr'):
        tds = row.find_all('td')
        if not tds:
            continue
        row_data = []
        # Procesar cada celda
        for i, td in enumerate(tds):
            # Para las columnas Col9 a Col15, evaluar el estilo
            if i in range(8, 15):  # índices 8 a 14 (Col9 a Col15)
                style = td.get("style", "")
                if "background-color:LightGrey" in style:
                    row_data.append(True)
                else:
                    row_data.append(False)
            else:
                row_data.append(td.get_text(strip=True))
        if len(row_data) != 18:
            print("Fila con cantidad inesperada de columnas (se esperan 18):", row_data)
            continue  # Se omite la fila si no cumple la cantidad esperada
        rows.append(row_data)

    # Definir manualmente los encabezados (ajusta los nombres según corresponda)
    headers = [
        "Código", "Materia", "Sede", "ID", "Modalidad",
        "Idioma", "Turno", "Año",
        "LUNES", "MARTES", "MIERCOLES", "JUEVES", "VIERNES", "SABADO", "DOMINGO",
        "Horario", "Fechas", "Tipo"
    ]

    return headers, rows

def filter_materias(df, url_filtro, threshold=0.4):
    """
    Filtra el DataFrame 'df' dejando únicamente las materias que sean similares a
    alguna de las materias listadas en la URL de filtro (CSV).

    Se ignora la columna B del CSV y se utiliza únicamente la columna A. Tanto
    los nombres en el CSV como los del DataFrame se normalizan (quitando espacios y
    pasando a minúsculas). Se utiliza difflib para comparar de forma aproximada,
    aceptándose aquellas coincidencias cuyo ratio sea mayor o igual a 'threshold'.

    Si url_filtro es una cadena vacía, no se aplica ningún filtro.
    """
    if not url_filtro.strip():
        print("No se aplica filtro (url_filtro vacía).")
        return df

    try:
        # Leer el CSV desde la URL
        filtro_df = pd.read_csv(url_filtro)
        # Usar solo la primera columna (columna A) y limpiar espacios y pasar a minúsculas
        lista_materias = filtro_df.iloc[:, 0].astype(str).str.strip().str.lower().tolist()
        print(f"Filtrando materias: se encontraron {len(lista_materias)} materias en la lista de filtro.")

        # Normalizar la columna "Materia" del DataFrame
        df["Materia_norm"] = df["Materia"].astype(str).str.strip().str.lower().str.replace('"', '').str.replace('"', '').apply(eliminar_tildes)

        # Función que verifica si la materia es similar a alguna materia en la lista
        def is_similar(materia):
            for m in lista_materias:
                ratio = difflib.SequenceMatcher(None, materia, m).ratio()
                if ratio >= threshold:
                    return True
            return False

        # Aplicar la función a cada fila para determinar si se incluye
        df_filtrado = df[df["Materia_norm"].apply(is_similar)].copy()
        df_filtrado.drop(columns="Materia_norm", inplace=True)
        return df_filtrado
    except Exception as e:
        print("Error al filtrar materias:", e)
        return df

# Iniciar sesión
session = requests.Session()

# Primera carga: obtener parámetros iniciales y HTML completo
response = session.get(url, headers=http_headers)
soup = BeautifulSoup(response.text, 'html.parser')

# (Opcional) Guardar la página inicial para revisar su estructura
with open('pagina_inicial.html', 'w', encoding='utf-8') as f:
    f.write(response.text)

# Obtener los campos del formulario
form_data = get_form_fields(soup)

# Primera solicitud POST con los parámetros obtenidos
response = session.post(url, headers=http_headers, data=form_data)
soup = BeautifulSoup(response.text, 'html.parser')

# Procesar la primera "página" de datos
headers, data = parse_table(soup)
all_data = data.copy()

# Detectar paginación (si existe)
pagination = soup.find('div', class_='rgInfoPart')
total_pages = 1
if pagination:
    match = re.search(r'en (\d+) paginas?\(s\)', pagination.get_text(strip=True))
    if match:
        total_pages = int(match.group(1))

print(f"Total de páginas: {total_pages}")

# Recorrer las páginas restantes (si total_pages > 1)
for page in range(2, total_pages + 1):
    # Actualizar los campos del formulario desde la página actual
    form_data = get_form_fields(soup)

    # Actualizar parámetros para paginación (verificar que __EVENTTARGET sea el correcto)
    form_data.update({
        '__EVENTTARGET': 'ctl00$ContentPlaceHolderMain$RadGrid1$ctl00$ctl03$ctl01$ctl12',
        '__EVENTARGUMENT': '',
        '__LASTFOCUS': ''
    })

    # Solicitud POST para la siguiente página
    response = session.post(url, headers=http_headers, data=form_data)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extraer datos de la página
    _, page_data = parse_table(soup)
    if page_data:
        all_data.extend(page_data)
    else:
        print(f"No se encontraron filas válidas en la página {page}.")

    print(f"Procesada página {page}/{total_pages}")

# Crear DataFrame con los datos extraídos
if headers and all_data:
    try:
        df = pd.DataFrame(all_data, columns=headers)
        # Aplicar filtro si se ha especificado una url_filtro
        df = filter_materias(df, url_filtro, threshold=0.8)
        df.to_csv('materias_completas.csv', index=False, encoding='utf-8-sig')
        print("Archivo guardado correctamente: materias_completas.csv")
        print("Importar CSV a https://uade.netlify.app/")
    except Exception as e:
        print("Error al crear el DataFrame:", e)
else:
    print("No se encontraron datos válidos")


Total de páginas: 3
Procesada página 2/3
Procesada página 3/3
Filtrando materias: se encontraron 9 materias en la lista de filtro.
Archivo guardado correctamente: materias_completas.csv
Importar CSV a https://uade.netlify.app/
